In [2]:
professions_category_tokens = {
    'Медик' : [
      'лекарство',
      'больница',
      'боль',
      'пациент',
      'нежелательные',
      'явления',
      'лечение',
      'доктор',
      'врач',
      'проблемы',
      'здоровье',
      'лечение',
      'болезнь',
      'лекарство',
      'аптека',
      'таблетка',
      'иньекция',
    ],
    'Наука' : [
        "грант", 
        'читатель', 
        'публиковать',  
        'мероприятие', 
        'химия', 
        'наука',
        'химик',
        'лаборатория', 
        'технарь',
        'инженер',
        'университет',
        'институт',
        'отдел',
        'студент',
        'кампус',
        'кафедра', 
        'элемент',
        'соединение',
        'методика',
        'раствор',
        'кристалл'
    ],
    'Полицейский' : [
      'должностные',
      'должность',
      'пост',
      'инструкции',
      'документ',
      'процессуальное',
      'решение',
      'жалоба',
      'правопорядок',
      'охрана',
      'противодействие',
      'полиция',
      'борьба',
      'преступление',
      'кодекс',
    ],
    'In office' : [
    ],
    'Films, tv and radio' : [
    ],
    'Arts and culture' : [
    ],
    'Бизнес сфера' : [
      'деньги',
      'доход',
      'расход',
      'экономика',
      'доллары',
      'валюта',
      'актив',
      'договор',
      'кредит',
      'бизнес',
      'работники',
      'увольнение',
      'найм',
      'отдел',
      'экономисты',
      'менеджеры',
      'маркетинг'
    ],
    'Science' : [
    ],
    'Education in school' : [
    ],
    'Education in university' : [
    ],
    'Сфера услуг' : [
      'оплата',
      'услуг',
      'заказ',
      'выполнение',
      'обязательства',
      'доставка',
      'невыполнение',
      'организация',
      'работа',
      'клиент',
      'задача',
      'услуга',
    ],
    'Дизайн' : [
      'логотип',
      'креатив',
      'рисование',
      'макет',
      'дизайн',
      'создание'
    ],
    'Программирование' : [
      'разработчик',
      'проект',
      'код',
      'программа',
      'команда',
      'система',
      'программирование',
      'проект'
    ]
}

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
def forms(s):
    s = str(s).lower()
    res = []
    if s[-1]=='а':  # 1 склонение
        res.append(s)  # И
        res.append(s[:-1]+'ы')  # Р
        res.append(s[:-1]+'е')  # Д
        res.append(s[:-1]+'у')  # В
        res.append(s[:-1]+'ой')  # Т
        res.append(s[:-1]+'е')  # П
    if s[-1]=='я':  # 1 склонение
        res.append(s)  # И
        res.append(s[:-1]+'и')  # Р
        res.append(s[:-1]+'е')  # Д
        res.append(s[:-1]+'ю')  # В
        res.append(s[:-1]+'ей')  # Т
        res.append(s[:-1]+'е')  # П
    elif s[-1]=='о':  # 2 склонение (с окончанием)
        res.append(s)  # И
        res.append(s[:-1]+'а')  # Р
        res.append(s[:-1]+'у')  # Д
        res.append(s)  # В
        res.append(s[:-1]+'ом')  # Т
        res.append(s[:-1]+'е')  # П
    elif s[-1]=='е':  # 2 склонение (с окончанием)
        res.append(s)  # И
        res.append(s[:-1]+'я')  # Р
        res.append(s[:-1]+'ю')  # Д
        res.append(s)  # В
        res.append(s[:-1]+'ем')  # Т
        res.append(s[:-1]+'и')  # П
    elif s[-1]=='ь':  # 3 склонение
        res.append(s)  # И
        res.append(s[:-1]+'и')  # Р
        res.append(s[:-1]+'и')  # Д
        res.append(s[:-1]+'и')  # В
        res.append(s[:-1]+'ью')  # Т
        res.append(s[:-1]+'и')  # П
    else:  # если хз, какое склонение, добавляет сразу несколько
           # лучше, чем ничего
        res.append(s)
        res.append(s[:-1]+'и')
        res.append(s[:-1]+'е')
        res.append(s[:-1]+'а')
        res.append(s[:-1]+'у')
        res.append(s[:-1]+'ы')
    return list(set(res))

In [11]:
def category_percent(token_vocabulary):
    # создаем базу тег - професиия
    X = []  # тег
    Y = []  # профессия
    for proff, tag in professions_category_tokens.items():
        X += tag
        for i in range(len(tag)):
            Y.append(proff)
    
    # создаем базу падежи и теги профессии - профессия
    Xp = []
    Yp = []
    for i in range(len(X)) :
        f = forms(X[i])
        Xp += f
        Yp += [Y[i]] * len(f)
    
    vectorizer = CountVectorizer()  # векторизируем текст Xp
    Xp = vectorizer.fit_transform(Xp)
    
    clf = LogisticRegression()  # обучаем нейросеть
    clf.fit(Xp,Yp)
    
    # сортируем пары ключ - значение по максимальному значению
    keys = []  
    values = []
    for key, value in token_vocabulary.items():
        keys.append(key)
        values.append(int(value*100))
    values_sorted = sorted(values, reverse=True)

    sum_values = 0
    keys_sorted = []
    values_sorted_persent = []
    for i in range(len(values_sorted)):
        sum_values += values_sorted[i]
        keys_sorted.append(keys[values.index(values_sorted[i])])
    for i in range(len(values_sorted)):
        values_sorted_persent.append (int(values_sorted[i] * 10000/sum_values))
        
    keys_sorted_searched = []  # в эту переменную кладутся профессии от нейронки
    for i in range(len(keys_sorted)):
        s = keys_sorted[i]
        s = (s.lower().split())
        s = vectorizer.transform(s).toarray()
        keys_sorted_searched.append(clf.predict(s)[0])
        
    dict_i = {}
    for i in range(len(keys_sorted_searched)):
        dict_i.update({keys_sorted_searched[i] : values_sorted_persent[i]})
    return dict_i

In [8]:
token_vocabulary = {
    'лечение' : 0.4,
    'заказ' : 0.3,
    'разработка' : 0.7
}

In [9]:
token_vocabulary = {'пикаба': 1, 'фабрика': 1, 'день': 1, 'хотеть': 1, 'проводить': 1, 'время': 2, 'интерес': 1, 'пост': 2, 'проходить': 1, 'отбор': 1, 'пользователь': 1, 'добавлять': 2, 'сайт': 1, 'помощь': 2, 'плюс': 1, 'читатель': 1, 'поднимать': 1, 'отбирать': 1, 'публиковать': 1, 'группа': 2, 'читать': 2, 'тысяча': 1, 'человек': 1, 'комментарий': 1, 'приложение': 1, 'психология': 1, 'материал': 1, 'сеть': 2, 'кот': 1, 'жизнь': 2, 'рай': 1, 'любитель': 1, 'кошка': 1, 'юмор': 3, 'сообщество': 2, 'ценить': 1, 'формировать': 1, 'взгляд': 1, 'реальность': 1, 'призма': 1, 'ирония': 1, 'сатира': 1, 'сарказм': 2, 'факт': 1, 'первое': 1, 'вконтакте': 1, 'изобличение': 1, 'москва': 1, 'дом': 1, 'любим': 1, 'место': 2, 'событие': 1, 'закрываться': 1, 'отменяться': 1, 'переноситься': 1, 'просить': 1, 'паника': 1, 'ситуация': 1, 'улучшиться': 1, 'конец': 1, 'менять': 1, 'принцип': 1, 'работа': 2, 'продолжить': 1, 'оффлайна': 1, 'мероприятие': 1, 'стать': 2, 'развлечение': 1, 'активность': 1, 'создать': 1, 'спецпроект': 1, 'метр': 1, 'найда': 1, 'информация': 1, 'отмена': 1, 'перенос': 1, 'концерт': 1, 'шоу': 1, 'фестиваль': 1, 'ждать': 2, 'совет': 1, 'отдых': 1, 'прогулка': 1, 'музей': 1, 'трансляция': 1, 'спектакль': 1, 'экскурсия': 1, 'разбер': 1, 'рассказать': 1, 'осторожность': 1, 'образ': 1, 'беречь': 1, 'грустить': 1, 'живить': 1, 'понимание': 1, 'команда': 1, 'стих': 1, 'великое': 1, 'поэт': 1, 'литература': 1, 'мемделей': 1, 'химия': 2, 'царица': 1, 'наука': 1, 'собираться': 1, 'химик': 2, 'тэг': 1, 'вопрос': 1, 'красота': 1, 'лаборатория': 1, 'рекомендовать': 1, 'батлы': 1, 'новость': 2, 'словарь': 1, 'анло': 1, 'аудиословарь': 1, 'подписчик': 2, 'кинопоиск': 1, 'кино': 1, 'сериал': 1, 'билет': 1, 'появляться': 1, 'слух': 1, 'фильм': 1, 'превращаться': 1, 'показывать': 1, 'постер': 1, 'тизер': 1, 'трейлер': 1, 'нажимать': 1, 'быть': 2, 'рецензия': 1, 'покупать': 1, 'ставить': 1, 'оценка': 1, 'писать': 2, 'отзыв': 1, 'раз': 1, 'пересматривать': 1, 'уходить': 1, 'архив': 1, 'психопат': 1, 'создавать': 2, 'позитив': 1, 'химичка': 1, 'люба': 1, 'душа': 1, 'авантюрист': 1, 'псих': 1, 'технарь': 2, 'правило': 1, 'ругаться': 1, 'мат': 1, 'господин': 1, 'уважать': 1, 'друг': 1, 'ветка': 1, 'обсуждение': 1, 'помнить': 1, 'изготовление': 1, 'применение': 1, 'том': 1, 'наркотик': 1, 'взрывчатка': 1, 'паблик': 1, 'стоить': 1, 'оффтоп': 1, 'троллинг': 1, 'флуда': 1, 'предоставление': 1, 'решение': 1, 'помочь': 1, 'атмосфера': 1, 'дружба': 1, 'добро': 1, 'довериться': 1, 'инженер': 1, 'решать': 1, 'проблема': 1, 'генератор': 1, 'ахмеджинер': 1, 'гений': 1, 'девайс': 1, 'эдвайс': 1, 'альбом': 1, 'статья': 1, 'давать': 1, 'дело': 1, 'цех': 1, 'беседа': 1, 'психотерапия': 1, 'управление': 1, 'освещение': 1, 'деятельность': 1, 'подразделение': 1, 'мирэ': 1, 'университет': 1, 'сокращ': 1, 'увиср': 1, 'включать': 1, 'отдел': 1, 'программа': 1, 'общежитие': 1, 'центр': 1, 'культура': 1, 'творчество': 1, 'карьера': 1, 'служба': 1, 'поддержка': 1, 'студент': 1, 'приём': 1, 'обед': 1, 'график': 1, 'кампус': 1, 'мочь': 1, 'адрес': 1, 'кафедра': 1, 'митхта': 1, 'дисциплина': 1, 'основа': 1, 'элемент': 1, 'соединение': 1, 'методика': 1, 'преподавание': 1, 'равновесие': 1, 'раствор': 1, 'др': 1, 'направление': 1, 'кристалл': 1, 'использование': 1, 'технология': 1, 'обучение': 1}

In [12]:
print(category_percent(token_vocabulary))

/home/alexandr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/alexandr/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


{'Наука': 39}
